In [302]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from datetime import datetime, time
import pandas_datareader.data as pdr
import yfinance as yf 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

In [303]:
def stock_data(ticker, start, end):
    try:
        prices = yf.download(ticker, start, end, auto_adjust=True)['Close']
    except Exception as e:
        print(f"Error: {e}")
    return prices
    

In [394]:
class DualMomentum:
    def __init__(self,
                 prices = pd.DataFrame, market_prices = pd.DataFrame, mom_lookback=126, ma_window=200, top_n=5):
        """prices : daily prices of stocks 
        market_prices : daily prices of market index"""
        
        self.prices = prices
        self.market_prices = market_prices
        self.mom_lookback = mom_lookback
        self.ma_window = ma_window
        self.top_n = top_n

    def dual_momemtum(self):
        """Calculating 6 month dual momentum with monthly rebalancing"""


        #Monthly rebalancing to trade monthly
        prices_rb = self.prices.resample("ME").last()
    
   

        #Absolute Momemtum for the market
        abs_mom = self.market_prices.pct_change(self.mom_lookback)
        trend = self.market_prices > self.market_prices.rolling(self.ma_window).mean()
        market_signal = (abs_mom > 0) & trend
        market_signal_rb = market_signal.reindex(prices_rb.index)

        #print(market_signal.mean()) #debug

        #Relative momentum for stocks
        stocks_return = self.prices.pct_change(self.mom_lookback).reindex(prices_rb.index)
        

        #Ranking stocks
        global ranks
        ranks = stocks_return.rank(axis = 1, ascending= False, method='first')
        top_stocks = ranks <= self.top_n
   
        #monthly weights
        weights_rb = pd.DataFrame(0.0, index= prices_rb.index, columns=self.prices.columns)
        "this step is initializing weight matrix. rows are rebalance dates . Cash is represented as all zeros"

        for date in prices_rb.index:
            if market_signal_rb.loc[date].item():
                selected = top_stocks.loc[date]
                if selected.sum() > 0:
                    weights_rb.loc[date, selected.index] = (selected / selected.sum()) #assigning equal weight to selected stocks

        #Daily tradable weights
        global weights
        weights = ( weights_rb.reindex(self.prices.index).ffill().shift(1).fillna(0))

        """  reindex -> expands weights from monthly to daily and add Nads to non-rebalace days.
            ffill() - forward-fills last rebelance weights and portfolio is held constant between rebalance dates
            shift(1) - signals use today's close price (prevent lookahead bias)
            fillna(0) - before first signal. everything is in cash which is presented by 0.  """

        #Strategy returns
        returns = self.prices.pct_change()
        global strategy_returns 
        strategy_returns =( weights * returns).sum(axis = 1)
        global equity_curve
        equity_curve = (1+ strategy_returns).cumprod()

        #print(weights.sum(axis=1).mean()) #debug
        #print(strategy_returns.std()) #debug
        

        #Market return
        global market_return
        market_return = self.market_prices.pct_change()

        #Market signal daily
        global market_signal_daily
        market_signal_daily = (market_signal.reindex(equity_curve.index)
                               .ffill().fillna(False).squeeze()
                                .astype(bool))
       
       

    def performance_metrics(self):
        "Performance metrics"
        #Annualised return - cagr -> compounded yearly growth rate of the startegy
        trading_days = 252
        cagr = ( (1 + strategy_returns).prod()** (trading_days / len(strategy_returns)) -1 )
        cagr_market = ((1+ market_return).prod().iloc[-1]** (trading_days / len(market_return))-1 )
       

        #Annulised Volatility
        volatilty = strategy_returns.std() * np.sqrt(trading_days)

        #Sharpe Ratio
        sharpe_ratio = ( strategy_returns.mean() /strategy_returns.std()) * np.sqrt(trading_days)

        #maximum Drawdown
        rolling_max = equity_curve.cummax()
        drawdown = (equity_curve / rolling_max )-1
        max_drawdown = drawdown.min()

        #Calmer ratio -> return per unit drawdown
        calmer = cagr/abs(max_drawdown)

        win_rate = (strategy_returns >0).mean()

        turnover = weights.diff().abs().sum(axis=1).mean()

        #Summary table for performance

        data = pd.Series( { 
            "CAGR" : cagr,
            "Market CAGR": cagr_market,
            "Volatility" : volatilty,
            "Sharpe Ratio" : sharpe_ratio,
            "max Drawdown" : max_drawdown,
            "Calmer Ratio" : calmer,
            "Win rate" : win_rate,
            "Turnover": turnover
             }, name ='Metrics')
        
        performance = data.to_frame().reset_index()
        
        return performance


    def plot_return(self):
        """Plot the equity curve and drawdown"""
        
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=equity_curve.index, y = equity_curve, mode="lines", name='Dual Momentum'))

        fig.update_layout(title = "Dual Momentum Equity curve",
                      xaxis_title="Date", yaxis_title ='Portfolio value', yaxis_type = "log",
                      template="plotly_white", hovermode ='x unified')
        fig.show()

        #Drawdown
        rolling_max = equity_curve.cummax()
        drawdown = (equity_curve / rolling_max )-1

        fig1 = go.Figure()
        fig1.add_trace(go.Scatter(x = drawdown.index, y = drawdown, fill ='tozeroy', name = 'Drawdown'))
        fig1.update_layout(title='Drawdown', xaxis_title = 'Date', yaxis_title = 'Drawdown', 
                           template='plotly_white',hovermode ='x unified')
        fig1.show()
    

    def plot_market_regime(self, top_n):
        """Plotting Market Regime with respect to equity curve
        Also Plotting ranks of the stocks"""
        
        fig1 = go.Figure()
    
        for col in ranks.columns:
            fig1.add_trace(go.Scatter(x=ranks.index,
                                 y=ranks[col],
                                 mode="lines", name=col))
        
        fig1.update_layout( height=800, width =1000, autosize=False,
                            title="Momentum Rank", xaxis_title='Date', 
                            yaxis_title="Rank(1=Best)", yaxis_autorange="reversed",
                            template ="plotly_white", hovermode="x unified")
    
        fig1.add_hrect(y0=0.5, y1=top_n+0.5, 
                       fillcolor="green", opacity=0.15,
                   layer="below", line_width=0)
    
        fig1.show()


        #Equity curve and Market regime
        fig = go.Figure()

        fig.add_trace(
                    go.Scatter(x=equity_curve.index,  y=equity_curve.values,
                    name="Equity Curve",
                        line=dict(width=2)))
                                                   

        # -------------------------
        # Risk-on regime (clean boolean Series)
        # -------------------------
        risk_on = (
                market_signal_daily
                .reindex(equity_curve.index)
                .ffill()
                .fillna(False)
                .squeeze()
                .astype(bool)
                    )

        # -------------------------
        # Identify regime start / end (vectorized)
        # -------------------------
        regime_change = risk_on.astype(int).diff().fillna(0)

        starts = regime_change[regime_change == 1].index
        ends   = regime_change[regime_change == -1].index

        # Handle edge case: regime active at end
        if len(starts) > len(ends):
            ends = ends.append(pd.Index([risk_on.index[-1]]))

        # -------------------------
        # Add shaded risk-on regions
        # -------------------------
        for start, end in zip(starts, ends):
                fig.add_vrect(
                x0=start,
                x1=end,
                fillcolor="green",
                opacity=0.15,
                layer="below",
                line_width=0)
    

        # -------------------------
        # Layout
        # -------------------------
        fig.update_layout(
            title="Equity Curve with Market Regime (Risk-On Shaded)",
            template="plotly_white",
            hovermode="x unified")

        fig.show()



        
    

        


In [395]:
stocks = ['LLY', 'JNJ', 'AMGN','MRK','NVO','PFE']
market ='SPY'

In [396]:
prices = stock_data(stocks, "2010-01-01","2025-12-31")
prices.tail(10)

[*********************100%***********************]  6 of 6 completed


Ticker,AMGN,JNJ,LLY,MRK,NVO,PFE
Date,,,,,,
2025-12-16,326.739990,209.300003,1054.290039,98.269997,48.959999,25.530001
2025-12-17,326.010010,210.330002,1041.790039,99.180000,47.770000,25.040001
2025-12-18,324.420013,208.309998,1056.880005,100.690002,47.610001,25.040001
2025-12-19,327.380005,206.369995,1071.439941,101.089996,48.090000,25.190001
2025-12-22,331.390015,207.320007,1076.479980,104.720001,48.099998,25.209999
2025-12-23,331.489990,205.779999,1071.640015,105.040001,51.610001,24.879999
2025-12-24,333.959991,207.779999,1076.979980,106.449997,52.560001,25.030001
2025-12-26,332.929993,207.630005,1077.750000,106.779999,52.400002,25.090000
2025-12-29,329.630005,207.559998,1078.729980,106.620003,51.470001,25.000000


In [397]:
market_prices = stock_data(market, "2010-01-01", "2025-12-31")
market_prices.tail(10)

[*********************100%***********************]  1 of 1 completed


Ticker,SPY
Date,
2025-12-16,676.869934
2025-12-17,669.421936
2025-12-18,674.476929
2025-12-19,680.590027
2025-12-22,684.830017
2025-12-23,687.960022
2025-12-24,690.380005
2025-12-26,690.309998
2025-12-29,687.849976


In [398]:
strategy = DualMomentum(prices, market_prices, 126,200,5)

In [399]:
strategy.dual_momemtum()

In [400]:
metrics = strategy.performance_metrics()
metrics

,index,Metrics
0,CAGR,0.129144
1,Market CAGR,0.139828
2,Volatility,0.142953
3,Sharpe Ratio,0.921138
4,max Drawdown,-0.216360
5,Calmer Ratio,0.596896
6,Win rate,0.402933
7,Turnover,0.008998


In [401]:
strategy.plot_market_regime(3)

In [402]:
strategy.plot_return()

Tech Stocks

In [403]:
stocks = ['AMD', 'TSM', "NVDA",  'GOOG','MSFT']
market ='SPY'


In [404]:
prices = stock_data(stocks, "2000-01-01","2025-12-31")
prices.tail(10)

[*********************100%***********************]  5 of 5 completed


Ticker,AMD,GOOG,MSFT,NVDA,TSM
Date,,,,,
2025-12-16,209.169998,307.730011,476.390015,177.720001,286.869995
2025-12-17,198.110001,298.059998,476.119995,170.940002,276.959991
2025-12-18,201.059998,303.750000,483.980011,174.139999,284.679993
2025-12-19,213.429993,308.609985,485.920013,180.990005,288.950012
2025-12-22,214.949997,311.329987,484.920013,183.690002,293.279999
2025-12-23,214.899994,315.679993,486.850006,189.210007,296.950012
2025-12-24,215.039993,315.670013,488.019989,188.610001,298.799988
2025-12-26,214.990005,314.959991,487.709991,190.529999,302.839996
2025-12-29,215.610001,314.390015,487.100006,188.220001,300.920013


In [405]:
market_prices = stock_data(market,"2000-01-01", "2025-12-31" )

[*********************100%***********************]  1 of 1 completed


In [406]:
strategy_tech = DualMomentum(prices,market_prices, 126,200, 3)

In [407]:
strategy_tech.dual_momemtum()

In [424]:
metrics = strategy_tech.performance_metrics()
metrics

,index,Metrics
0,CAGR,0.159659
1,Market CAGR,0.080750
2,Volatility,0.231083
3,Sharpe Ratio,0.756919
4,max Drawdown,-0.423426
5,Calmer Ratio,0.377066
6,Win rate,0.375497
7,Turnover,0.003273


In [409]:
strategy_tech.plot_market_regime(3)

In [410]:
strategy_tech.plot_return()

Senstivity Analysis

In [439]:
#Grid Search structure
class SensitivityAnalysis:
    def __init__(self, DualMomentum,prices,market_prices):
        self.DualMomentum = DualMomentum
        self.prices = prices
        self.market_prices = market_prices
    
    def run_analysis(self, mom_lb =[126,189,252],
                     ma_lb = [150, 200,250],
                     top_n = [3,5]):

        result = []
        

        for mom in mom_lb: #6,9,12 months
            for ma in ma_lb:
                for n in top_n:
                
                    strategy_sen_analysis = self.DualMomentum(self.prices, self.market_prices,mom,ma,n)
                    strategy_sen_analysis.dual_momemtum()
                    metrics = strategy_sen_analysis.performance_metrics()
                    metrics_needed = ['CAGR', 'Sharpe Ratio', 'Calmer Ratio']
                    metrics_dict = (metrics.set_index("index")["Metrics"]
                                    .loc[metrics_needed].to_dict())
                    
                    result.append({
                            "mom_lb": mom,
                            "trend_ma": ma,
                            "top_n": n,
                            **metrics_dict
                        })
                    
                    
        return pd.DataFrame(result)



            


In [441]:
analysis = SensitivityAnalysis(DualMomentum, prices,market_prices)
df_sens = analysis.run_analysis()

In [452]:
df_sens[df_sens["top_n"]==3]

,mom_lb,trend_ma,top_n,CAGR,Sharpe Ratio,Calmer Ratio
0,126,150,3,0.185014,0.769935,0.391674
2,126,200,3,0.183865,0.771382,0.389241
4,126,250,3,0.189682,0.792297,0.415302
6,189,150,3,0.181162,0.762739,0.364020
8,189,200,3,0.187773,0.778022,0.368820
10,189,250,3,0.183846,0.752548,0.361106
12,252,150,3,0.164285,0.729078,0.366898
14,252,200,3,0.165418,0.729763,0.369429
16,252,250,3,0.166908,0.719577,0.378508


Plotting Heatmap for various Metrics

In [ ]:
# ---------------------------------------
#  CAGR
# ---------------------------------------
df_plot = df_sens[df_sens["top_n"] == 3]

heatmap_data = df_plot.pivot(index = 'trend_ma',
                             columns="mom_lb",
                             values="CAGR")

fig = go.Figure(go.Heatmap(
    z=heatmap_data.values,
    x=heatmap_data.columns,
    y=heatmap_data.index,
    colorscale="RdYlGn",
    colorbar=dict(title='CAGR'), zmin=0.15,zmax=0.19
))
fig.update_layout(title="Dual Momentum Sensititvity Heatmap(CAGR)",
                  xaxis_title="Momentum Lookback",
                  yaxis_title='MA Lookback',
                  template='plotly_white'
)
fig.show()


In [ ]:
# ---------------------------------------
#  Calmer Ratio
# ---------------------------------------
df_plot = df_sens[df_sens["top_n"] == 3]

heatmap_data = df_plot.pivot(index = 'trend_ma',
                             columns="mom_lb",
                             values="Calmer Ratio")

fig = go.Figure(go.Heatmap(
    z=heatmap_data.values,
    x=heatmap_data.columns,
    y=heatmap_data.index,
    colorscale="RdYlGn",
    colorbar=dict(title='Calmer Ratio'), zmin=0.35,zmax=0.55
))
fig.update_layout(title="Dual Momentum Sensititvity Heatmap(Calmer ratio)",
                  xaxis_title="Momentum Lookback",
                  yaxis_title='MA Lookback',
                  template='plotly_white'
)
fig.show()

In [ ]:
# ---------------------------------------
#  Sharpe Ratio
# ---------------------------------------
df_plot = df_sens[df_sens["top_n"] == 3]

heatmap_data = df_plot.pivot(index = 'trend_ma',
                             columns="mom_lb",
                             values="Sharpe Ratio")

fig = go.Figure(go.Heatmap(
    z=heatmap_data.values,
    x=heatmap_data.columns,
    y=heatmap_data.index,
    colorscale="RdYlGn",
    colorbar=dict(title='Sharpe Ratio'), zmin=0.7,zmax=0.8
))
fig.update_layout(title="Dual Momentum Sensititvity Heatmap(Sharpe Ratio)",
                  xaxis_title="Momentum Lookback",
                  yaxis_title='MA Lookback',
                  template='plotly_white'
)
fig.show()